# Cопоставление геоназваний с унифицированными именами geonames

# Задача

- Создать решение для подбора наиболее подходящих названий с geonames.
Например Ереван -> Yerevan
- На примере РФ и стран наиболее популярных для релокации - Беларусь, Армения,
Казахстан, Кыргызстан, Турция, Сербия. Города с населением от 15000 человек (с
возможностью масштабирования на сервере заказчика)
- Возвращаемые поля *geonameid, name, region, country, cosine similarity*
- формат данных на выходе: список словарей, например [{dict_1}, {dict_2}, …. {dict_n}], где словарь - одна запись с указанными полями

# Цель

Сопоставление произвольных гео названий с унифицированными именами geonames для внутреннего использования Карьерным центром.

# План работы


- Подготовить данные к обучению.

 - Метод векторизации слова CountVectorizer();
 - Метод использования готовой предобученной модели из SentenceTransformers;
 - Метод использования своей предобученной модели из tansformers.

# Описание данных

Используемые таблицы с geonames:
- admin1CodesASCII
- alternateNamesV2
- cities15000
- countryInfo
- при необходимости любые другие открытые данные
- таблицы geonames можно скачать здесь http://download.geonames.org/export/dump/
- Тестовый датасет: https://disk.yandex.ru/d/wC296Rj3Yso2AQ


In [1]:
%load_ext autoreload
%autoreload 2

In [62]:
!pip freeze > requirements.txt

**Импортируем необходимые библиотеки (pandas, numpy и другие).**

In [2]:
import pandas as pd
import numpy as np

from count_vec_mod import CountVec
from semantic_mod import SemSearch
from generation_semantic_mod import GenSearch
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import notebook 

# NLP
import torch
import transformers
import nltk
import re
from sentence_transformers import SentenceTransformer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

In [3]:
!pip install pymystem3
from pymystem3 import Mystem

**Создадим локальную базу данных и подсоединимся к ней.**

In [4]:
# database connection
!pip install SQLAlchemy
!pip install --pre SQLAlchemy
!pip install psycopg2

import sqlalchemy
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

DATABASE = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'password\'',
    'host': 'localhost',
    'port': 5433,
    'database': 'postgres',
    'query': {}
}

engine = create_engine(URL(**DATABASE))

C:\Users\User\AppData\Local\Temp\ipykernel_18740\1662055909.py:21: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  engine = create_engine(URL(**DATABASE))


### Загрузка в базу данных таблиц

**Из предварительно загруженных файлов из geonames.org считаем информацию и загрузим в базу данных.**

#### 1. Возьмем таблицу 'alternateNamesV2'

In [5]:
alternateNamesV2 = pd.read_csv("C:/Users/User/Desktop/DS Python/Geonames/data/alternateNamesV2.txt", 
                               delimiter='\t', 
                               header=None,
                               names = [
                               'geonameid', 
                               'isoLanguage', 
                               'alternateName', 
                               'isPreferredName', 
                               'isShortName', 
                               'isColloquial', 
                               'isHistoric',
                               'from_period', 
                               'to_period'    
                               ])

C:\Users\User\anaconda3\envs\practicum\lib\site-packages\IPython\core\interactiveshell.py:3194: DtypeWarning: Columns (8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Познакомимся с таблицей.

In [6]:
alternateNamesV2.tail()

,geonameid,isoLanguage,alternateName,isPreferredName,isShortName,isColloquial,isHistoric,from_period,to_period
17666269,12628281,NaN,Minami-Shimoji,NaN,NaN,NaN,NaN,NaN,NaN
17666270,12628282,NaN,Nakanōgan Hill,NaN,NaN,NaN,NaN,NaN,NaN
17666271,12628282,NaN,Nakanougan Hill,NaN,NaN,NaN,NaN,NaN,NaN
17666272,12628283,NaN,Okisakishima Ridge,NaN,NaN,NaN,NaN,NaN,NaN
17666273,12628283,NaN,Oki-Sakishima Ridge,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
alternateNamesV2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16035367 entries, 1284819 to 17666273
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   geonameid        int64  
 1   isoLanguage      object 
 2   alternateName    object 
 3   isPreferredName  float64
 4   isShortName      float64
 5   isColloquial     float64
 6   isHistoric       float64
 7   from_period      object 
 8   to_period        object 
dtypes: float64(4), int64(1), object(4)
memory usage: 1.2+ GB


Оценим количество пропусков в данных

In [8]:
pd.DataFrame(round(alternateNamesV2.isna().mean()*100,2)).style.background_gradient('coolwarm')

,0
geonameid,0.000000
isoLanguage,42.650000
alternateName,0.000000
isPreferredName,96.620000
isShortName,99.280000
isColloquial,99.980000
isHistoric,99.820000
from_period,99.980000
to_period,99.980000


В столбцах много пропусков, проверим с чем связано.

In [9]:
alternateNamesV2['isShortName'].unique()

array([nan,  1.])

In [10]:
alternateNamesV2['isColloquial'].unique()

array([nan,  1.])

In [11]:
alternateNamesV2['isHistoric'].unique()

array([nan,  1.])

In [12]:
alternateNamesV2['isPreferredName'].unique()

array([nan,  1.])

Вероятнее всего, заполнялись только значения равные 1.0. Чтобы не потерять данные и не удалять строки, все пустые строки заполняю значением 0.0.

In [13]:
alternateNamesV2[['isShortName', 'isColloquial', 'isHistoric', 'isPreferredName']] = alternateNamesV2[[
    'isShortName', 'isColloquial', 'isHistoric', 'isPreferredName']].fillna(0.0, inplace=False)

In [14]:
pd.DataFrame(round(alternateNamesV2.isna().mean()*100,2)).style.background_gradient('coolwarm')

,0
geonameid,0.000000
isoLanguage,42.650000
alternateName,0.000000
isPreferredName,0.000000
isShortName,0.000000
isColloquial,0.000000
isHistoric,0.000000
from_period,99.980000
to_period,99.980000


In [15]:
alternateNamesV2['isPreferredName'].unique()

array([0., 1.])

Занесем данные из таблицы в базу данных.

In [16]:
alternateNamesV2.to_sql(name='alternatename', if_exists = 'replace', con=engine)

#### 2. Возьмем таблицу 'countryInfo'

In [17]:
countryinfo = pd.read_csv("C:/Users/User/Desktop/DS Python/Geonames/data/countryInfo.txt", 
                               delimiter='\t', 
                               header=None,
                               skiprows=50, 
                               names = [
                               'country_code', 
                               'iso_alpha3', 
                               'iso_numeric', 
                               'fips_code', 
                               'country', 
                               'capital', 
                               'areainsqkm', 
                               'population', 
                               'continent', 
                               'tld', 
                               'currency_code', 
                               'currency_name', 
                               'phone', 
                               'postal_code_format', 
                               'postal_code_regex',
                               'languages',
                               'geonameid', 
                               'neighbours', 
                               'equivalent_fips_code'
                                   ])

Познакомимся с таблицей.

In [18]:
countryinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   country_code          251 non-null    object 
 1   iso_alpha3            252 non-null    object 
 2   iso_numeric           252 non-null    int64  
 3   fips_code             249 non-null    object 
 4   country               252 non-null    object 
 5   capital               246 non-null    object 
 6   areainsqkm            252 non-null    float64
 7   population            252 non-null    int64  
 8   continent             210 non-null    object 
 9   tld                   251 non-null    object 
 10  currency_code         251 non-null    object 
 11  currency_name         251 non-null    object 
 12  phone                 247 non-null    object 
 13  postal_code_format    162 non-null    object 
 14  postal_code_regex     162 non-null    object 
 15  languages             2

In [19]:
countryinfo.head(10)

,country_code,iso_alpha3,iso_numeric,fips_code,country,capital,areainsqkm,population,continent,tld,currency_code,currency_name,phone,postal_code_format,postal_code_regex,languages,geonameid,neighbours,equivalent_fips_code
0,AD,AND,20,AN,Andorra,Andorra la Vella,468.0,77006,EU,.ad,EUR,Euro,376,AD###,^(?:AD)*(\d{3})$,ca,3041565,"ES,FR",NaN
1,AE,ARE,784,AE,United Arab Emirates,Abu Dhabi,82880.0,9630959,AS,.ae,AED,Dirham,971,NaN,NaN,"ar-AE,fa,en,hi,ur",290557,"SA,OM",NaN
2,AF,AFG,4,AF,Afghanistan,Kabul,647500.0,37172386,AS,.af,AFN,Afghani,93,NaN,NaN,"fa-AF,ps,uz-AF,tk",1149361,"TM,CN,IR,TJ,PK,UZ",NaN
3,AG,ATG,28,AC,Antigua and Barbuda,St. John's,443.0,96286,NaN,.ag,XCD,Dollar,+1-268,NaN,NaN,en-AG,3576396,NaN,NaN
4,AI,AIA,660,AV,Anguilla,The Valley,102.0,13254,NaN,.ai,XCD,Dollar,+1-264,NaN,NaN,en-AI,3573511,NaN,NaN
5,AL,ALB,8,AL,Albania,Tirana,28748.0,2866376,EU,.al,ALL,Lek,355,####,^(\d{4})$,"sq,el",783754,"MK,GR,ME,RS,XK",NaN
6,AM,ARM,51,AM,Armenia,Yerevan,29800.0,2951776,AS,.am,AMD,Dram,374,######,^(\d{6})$,hy,174982,"GE,IR,AZ,TR",NaN
7,AO,AGO,24,AO,Angola,Luanda,1246700.0,30809762,AF,.ao,AOA,Kwanza,244,NaN,NaN,pt-AO,3351879,"CD,NA,ZM,CG",NaN
8,AQ,ATA,10,AY,Antarctica,NaN,14000000.0,0,AN,.aq,NaN,NaN,NaN,NaN,NaN,NaN,6697173,NaN,NaN
9,AR,ARG,32,AR,Argentina,Buenos Aires,2766890.0,44494502,SA,.ar,ARS,Peso,54,@####@@@,"^[A-Z]?\d{4}[A-Z]{0,3}$","es-AR,en,it,de,fr,gn",3865483,"CL,BO,UY,PY,BR",NaN


Оценим количество пропусков в данных

In [20]:
pd.DataFrame(round(countryinfo.isna().mean()*100,2)).style.background_gradient('coolwarm')

,0
country_code,0.400000
iso_alpha3,0.000000
iso_numeric,0.000000
fips_code,1.190000
country,0.000000
capital,2.380000
areainsqkm,0.000000
population,0.000000
continent,16.670000
tld,0.400000


Занесем данные из таблицы в базу данных.

In [21]:
countryinfo.to_sql(name='countryinfo', if_exists = 'replace', con=engine)

#### 3. Возьмем таблицу 'admin1CodesASCII'

In [22]:
admintocodes = pd.read_csv("C:/Users/User/Desktop/DS Python/Geonames/data/admin1CodesASCII.txt", 
                               delimiter='\t', 
                               header=None, 
                               names = [
                               'code', 
                               'region', 
                               'ascii_region', 
                               'geonameid'
                               ])

Познакомимся с таблицей.

In [23]:
admintocodes.head(10)

,code,region,ascii_region,geonameid
0,AD.06,Sant Julià de Loria,Sant Julia de Loria,3039162
1,AD.05,Ordino,Ordino,3039676
2,AD.04,La Massana,La Massana,3040131
3,AD.03,Encamp,Encamp,3040684
4,AD.02,Canillo,Canillo,3041203
5,AD.07,Andorra la Vella,Andorra la Vella,3041566
6,AD.08,Escaldes-Engordany,Escaldes-Engordany,3338529
7,AE.07,Imārat Umm al Qaywayn,Imarat Umm al Qaywayn,290595
8,AE.05,Raʼs al Khaymah,Imarat Ra's al Khaymah,291075
9,AE.03,Dubai,Dubai,292224


Оценим количество пропусков в данных

In [24]:
pd.DataFrame(round(admintocodes.isna().mean()*100,2)).style.background_gradient('coolwarm')

,0
code,0.000000
region,0.000000
ascii_region,0.000000
geonameid,0.000000


Занесем данные из таблицы в базу данных.

In [25]:
admintocodes.to_sql(name='admintocodes', if_exists = 'replace', con=engine)

#### 3. Возьмем таблицу 'cities15000'

In [26]:
cities15000 = pd.read_csv("C:/Users/User/Desktop/DS Python/Geonames/data/cities15000.txt", 
                               delimiter='\t', 
                               header=None,
                               names = [
                               'geonameid', 
                               'name', 
                               'asciiname', 
                               'alternatenames', 
                               'latitude', 
                               'longitude', 
                               'feature_class',
                               'feature_code',
                               'country_code', 
                               'cc2',
                               'admin1_code', 
                               'admin2_code', 
                               'admin3_code', 
                               'admin4_code', 
                               'population',
                               'elevation',
                               'dem', 
                               'timezone',
                               'modification_date' 
                               ])

Познакомимся с таблицей.

In [27]:
cities15000.head()

,geonameid,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
0,3040051,les Escaldes,les Escaldes,"Ehskal'des-Ehndzhordani,Escaldes,Escaldes-Engo...",42.50729,1.53414,P,PPLA,AD,NaN,08,NaN,NaN,NaN,15853,NaN,1033,Europe/Andorra,2008-10-15
1,3041563,Andorra la Vella,Andorra la Vella,"ALV,Ando-la-Vyey,Andora,Andora la Vela,Andora ...",42.50779,1.52109,P,PPLC,AD,NaN,07,NaN,NaN,NaN,20430,NaN,1037,Europe/Andorra,2020-03-03
2,290594,Umm Al Quwain City,Umm Al Quwain City,"Oumm al Qaiwain,Oumm al Qaïwaïn,Um al Kawain,U...",25.56473,55.55517,P,PPLA,AE,NaN,07,NaN,NaN,NaN,62747,NaN,2,Asia/Dubai,2019-10-24
3,291074,Ras Al Khaimah City,Ras Al Khaimah City,"Julfa,Khaimah,RAK City,RKT,Ra's al Khaymah,Ra'...",25.78953,55.94320,P,PPLA,AE,NaN,05,NaN,NaN,NaN,351943,NaN,2,Asia/Dubai,2019-09-09
4,291580,Zayed City,Zayed City,"Bid' Zayed,Bid’ Zayed,Madinat Za'id,Madinat Za...",23.65416,53.70522,P,PPL,AE,NaN,01,103,NaN,NaN,63482,NaN,118,Asia/Dubai,2019-10-24


In [28]:
cities15000['concat_code'] = cities15000['country_code'] + '.' + cities15000['admin1_code']

Оценим количество пропусков в данных

In [29]:
pd.DataFrame(round(cities15000.isna().mean()*100,2)).style.background_gradient('coolwarm')

,0
geonameid,0.000000
name,0.000000
asciiname,0.000000
alternatenames,8.510000
latitude,0.000000
longitude,0.000000
feature_class,0.000000
feature_code,0.000000
country_code,0.050000
cc2,99.950000


Занесем данные из таблицы в базу данных.

In [30]:
cities15000.to_sql(name='cities15000', if_exists = 'replace', con=engine)

### Выгрузка из базы данных требуемого запроса

In [31]:
query = '''SELECT c.geonameid, 
                  c.name, 
                  c.asciiname, 
                  c.alternatenames, 
                  c.population,
                  i.country,
                  c.country_code,
                  c.admin1_code,
                  c.concat_code,
                  a.region,  
                  a.ascii_region
           FROM cities15000 AS c
           LEFT JOIN countryinfo AS i ON c.country_code = i.country_code
           LEFT JOIN admintocodes AS a ON c.concat_code = a.code
           WHERE c.country_code in ('RU',
                                  'KZ',
                                  'BY',
                                  'AM',
                                  'KG',
                                  'RS',
                                  'TR');         
                                
'''


corpus_rus = pd.read_sql_query(query, con=engine)

Оценим нашу выгруженную таблицу.

In [32]:
corpus_rus.head()

,geonameid,name,asciiname,alternatenames,population,country,country_code,admin1_code,concat_code,region,ascii_region
0,616435,Masis,Masis,"Hrazdan,Masis,Narimanlu,Razdan,Takhanshalu,Tok...",18911,Armenia,AM,02,AM.02,Ararat,Ararat
1,174991,Ararat,Ararat,"Ararat,Araratas,Ararato,Davalinskiy Tsemzavod,...",28832,Armenia,AM,02,AM.02,Ararat,Ararat
2,174979,Artashat,Artashat,"Artachat,Artasat,Artasatas,Artasato,Artaschat,...",20562,Armenia,AM,02,AM.02,Ararat,Ararat
3,174972,Hats’avan,Hats'avan,"Acavan,Atsavan,Hats'avan,Hats’avan,Sisian,Ацав...",15208,Armenia,AM,08,AM.08,Syunik,Syunik
4,174895,Goris,Goris,"Geryusy,Goris,Горис,Գորիս",20379,Armenia,AM,08,AM.08,Syunik,Syunik


**Так как некоторые расчеты будут проводится с использование gpu, чтобы сразу загрузить данные на сервис и работать над задачей, сохраним таблицу.**

В файле *corpus.csv* будут храниться все выгруженные значения новой таблицы.

In [33]:
corpus_rus.to_csv(r"C:/Users/User/Desktop/DS Python/Geonames/data/corpus.csv", index=False, sep=";")

Сразу заменим пропуски на пустые строки.

In [34]:
corpus_rus['alternatenames'] = [name if name is not None else '' for name in corpus_rus['alternatenames']]

### Метод векторизации слова CountVectorizer()

Найдем расстояние между искомым словом и ближайшим словом из текста, приводя их к набору чисел.

Создадим экземпляр класса CountVectorizer().

In [35]:
# анализировать будем отдельные буквы и n-граммы
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2,4)) 

Создадим экземпляр кастомного класса CountVec().

In [36]:
vec_mod = CountVec()

Для проверки работы скрипта введем запрашиваемый город и количество ниболее близких названий городов. 

In [37]:
city = 'влодекасток' # запрашиваемый город

In [38]:
num = 3 # количество близких названий городов

Функция для подбора наиболее подходящих названий с geonames

In [39]:
top = vec_mod.get_cos(city, 
                       corpus_rus[['geonameid', 'asciiname', 'alternatenames', 'country', 'region']], 
                       vectorizer, 
                       num)

In [40]:
top

[{'geonameid': 2013348,
  'asciiname': 'Vladivostok',
  'region': 'Primorye',
  'country': 'Russia',
  'cosine_sim': 0.5773503184318542},
 {'geonameid': 743882,
  'asciiname': 'Kastamonu',
  'region': 'Kastamonu',
  'country': 'Turkey',
  'cosine_sim': 0.471404492855072},
 {'geonameid': 1520240,
  'asciiname': 'Pavlodar',
  'region': 'Pavlodar Region',
  'country': 'Kazakhstan',
  'cosine_sim': 0.471404492855072}]

### Метод использования предобученной модели из SentenceTransformers

Обработаем значения в таблице *corpus*

In [41]:
# Удаляем пропущенные значения из таблицы
corpus = corpus_rus.dropna()
# Разделяем значения в столбце 'alternatenames'
corpus.alternatenames = corpus.alternatenames.str.split(',')
# Приводим к виду 1 наименование asciiname = 1 наименование alternatenames
corpus = corpus.explode('alternatenames')
# Удаляем совпадающие значения в столбцах
corpus = corpus[corpus.asciiname!=corpus.alternatenames]
# Удаляем парные дубликаты из двух столбцов
corpus = corpus.drop_duplicates(subset=['asciiname', 'alternatenames'])

**Выведем первые 5 строк преобразованной таблицы и размер на экран.**

In [42]:
corpus.head()

,geonameid,name,asciiname,alternatenames,population,country,country_code,admin1_code,concat_code,region,ascii_region
0,616435,Masis,Masis,Hrazdan,18911,Armenia,AM,02,AM.02,Ararat,Ararat
0,616435,Masis,Masis,Narimanlu,18911,Armenia,AM,02,AM.02,Ararat,Ararat
0,616435,Masis,Masis,Razdan,18911,Armenia,AM,02,AM.02,Ararat,Ararat
0,616435,Masis,Masis,Takhanshalu,18911,Armenia,AM,02,AM.02,Ararat,Ararat
0,616435,Masis,Masis,Tokhanshalu,18911,Armenia,AM,02,AM.02,Ararat,Ararat


In [43]:
corpus.shape

(22741, 11)

Названия городов без дубликатов

In [44]:
names_list = corpus.asciiname.drop_duplicates().values
names_list

array(['Masis', 'Ararat', 'Artashat', ..., 'Bayburt', 'Akcakoca',
       'Duezce'], dtype=object)

**Выберем предварительно обученную модель из библиотеки SentenceTransformer и загрузим ее**

In [45]:
# Загрузка предварительно обученной модели mBART и токенизатора
model = SentenceTransformer('artefucktor/LaBSE_geonames_RU_RELOCATION')

Some weights of the model checkpoint at C:\Users\User/.cache\torch\sentence_transformers\artefucktor_LaBSE_geonames_RU_RELOCATION\ were not used when initializing BertModel: ['0.auto_model.encoder.layer.6.output.dense.weight', '0.auto_model.encoder.layer.7.attention.output.LayerNorm.bias', '0.auto_model.encoder.layer.10.attention.output.dense.bias', '0.auto_model.encoder.layer.3.intermediate.dense.weight', '0.auto_model.encoder.layer.8.intermediate.dense.bias', '0.auto_model.pooler.dense.weight', '0.auto_model.encoder.layer.8.output.LayerNorm.weight', '0.auto_model.encoder.layer.9.attention.output.LayerNorm.weight', '0.auto_model.encoder.layer.11.attention.self.value.bias', '0.auto_model.encoder.layer.4.attention.self.value.weight', '0.auto_model.encoder.layer.9.attention.output.LayerNorm.bias', '0.auto_model.encoder.layer.1.attention.output.LayerNorm.bias', '0.auto_model.encoder.layer.10.intermediate.dense.bias', '0.auto_model.encoder.layer.8.attention.self.key.weight', '0.auto_model.

Some weights of BertModel were not initialized from the model checkpoint at C:\Users\User/.cache\torch\sentence_transformers\artefucktor_LaBSE_geonames_RU_RELOCATION\ and are newly initialized: ['encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.7.attention.self.key.weight', 'pooler.dense.bias', 'encoder.layer.7.attention.self.value.weight', 'encoder.layer.1.attention.output.dense.weight', 'encoder.layer.2.attention.output.LayerNorm.bias', 'encoder.layer.6.attention.self.value.bias', 'encoder.layer.5.attention.self.query.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.2.attention.self.value.weight', 'encoder.layer.6.output.dense.bias', 'encoder.layer.6.attention.output.LayerNorm.bias', 'embeddings.word_embeddings.weight', 'encoder.layer.6.output.dense.weight', 'encoder.layer.7.attention.output.LayerNorm.weight', 'encoder.layer.6.attention.output.dense.weight', 'encoder.layer.7.attention.self.query.bias', 'encoder.layer.11.attention.self.value.weight', 'encod

Так как модель обучена на альтернативных именах, создадим эмбендинги для всех названий городов

In [46]:
embeddings = model.encode(names_list)
print(embeddings)

[[-0.01756764  0.06759417  0.0062893  ... -0.01650135 -0.06111738
   0.0183134 ]
 [-0.05108269  0.07602846  0.02132422 ... -0.01072385 -0.05783798
  -0.00085994]
 [-0.05927843  0.07991929  0.02733636 ... -0.00619107 -0.05208034
  -0.01652848]
 ...
 [-0.04823219  0.07744403  0.02462361 ... -0.0322054  -0.05414787
  -0.00470967]
 [-0.04080104  0.07947501  0.00854814 ... -0.02015225 -0.05453103
  -0.00165115]
 [-0.04958811  0.07369293  0.01528744 ... -0.02922699 -0.05832952
   0.00272034]]


Создадим экземпляр кастомного класса CountVec().

In [47]:
sem_mod = SemSearch()

Для проверки работы скрипта введем запрашиваемый город и количество ниболее близких названий городов.

In [48]:
city = 'влодекасток'

In [49]:
num = 3

Функция для подбора наиболее подходящих названий с geonames с помощью cosine-similarity

In [50]:
top_sim = sem_mod.get_sim(city, names_list, 
            embeddings, model, corpus_rus, top=3) 

In [51]:
top_sim

[{'geonameid': 174972,
  'asciiname': "Hats'avan",
  'region': 'Syunik',
  'country': 'Armenia',
  'score': 0.9614735841751099},
 {'geonameid': 1538636,
  'asciiname': "Novoural'sk",
  'region': 'Sverdlovsk Oblast',
  'country': 'Russia',
  'score': 0.9577018618583679},
 {'geonameid': 481350,
  'asciiname': 'Trubchevsk',
  'region': 'Bryansk Oblast',
  'country': 'Russia',
  'score': 0.9554844498634338}]

Функция для подбора наиболее подходящих названий с geonames с помощью mahalanobis

In [52]:
cov_matrix = np.cov(embeddings, rowvar=False)

In [53]:
top_mah = sem_mod.get_mahalanobis(city, names_list, 
                embeddings, model, corpus_rus, cov_matrix, top=3)

In [54]:
top_mah

[{'geonameid': 174972,
  'asciiname': "Hats'avan",
  'region': 'Syunik',
  'country': 'Armenia',
  'mahalanobis_distance': 0.006765640908900858},
 {'geonameid': 530849,
  'asciiname': 'Maloyaroslavets',
  'region': 'Kaluga Oblast',
  'country': 'Russia',
  'mahalanobis_distance': 0.007299732355036254},
 {'geonameid': 559654,
  'asciiname': 'Gorodishche',
  'region': 'Volgograd Oblast',
  'country': 'Russia',
  'mahalanobis_distance': 0.0073621988771283165}]

### Метод использования предобученной модели из tansformers

**Выберем свою предварительно обученную модель из библиотеки transformer и загрузим ее**

In [55]:
# Загрузка предварительно обученной модели mBART и токенизатора
output_model_path = "EldarKerimkhan/mbart-large-50-many-to-many-mmt.geonames_RU_RELOCATION"
model = MBartForConditionalGeneration.from_pretrained(output_model_path)
tokenizer = MBart50TokenizerFast.from_pretrained(output_model_path)

In [56]:
gen_mod = GenSearch()

Для проверки работы скрипта введем запрашиваемый город и количество ниболее близких названий городов.

In [57]:
city = 'Истанбул'

Функция для подбора наиболее подходящих названий с geonames

In [59]:
result = gen_mod.gen_sim_mbart(city, tokenizer, model, corpus_rus)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [60]:
result

{'geonameid': 745044,
 'asciiname': 'Istanbul',
 'region': 'Istanbul',
 'country': 'Turkey'}

## Выводы

В проекте была создана локальная база данных. В базу были загружены необходимые для выполнения задачи таблицы. А также запросом к базе данных выгружены данные, относящиеся к странам наиболее популярных для релокации. 

В данном файле были опробованы такие методы узнаваемости слова как:

 - Метод векторизации слова CountVectorizer();
 - Метод использования готовой предобученной модели из SentenceTransformers;
 - Метод использования своей предобученной модели из tansformers.

Для каждого метода была проведена оценка качества в файле `geonames _test_file`. 
В целом, все методы дают результаты узнаваемости более 80%. 

Метод векторизации слова `CountVectorizer()` показал результаты хуже на 5%, чем другие два метода. 
Готовая модель из SentenceTransformers была предобученна на 5 эпохах.
Своя модель была предобучена на 4 эпохах с использованием аугментации (случайных опечаток в слове Random Insertion)).

Очень важно сделать *правильную предобработку исходных слов*, в том числе добавить дополнительные слова с другими аугментациями удаление символов, перестановка символов, семантические аугментации, расширение синонимами.

В любом случае, все эти методы очень сильно зависят от столбца с *альтернативными названиями*. То есть данный словарь должен пополняться, если встречаются новые слова. (но пополняться осторожно, так как нужно различать, насколько данное слово подходит в качестве альтернативного названия). 